## Download Dataset

In [1]:
# download dataset (etd 10mins)
!wget -nc "https://zenodo.org/records/17351690/files/imagine_decoding_challenge.zip?download=1" -O imagine_decoding_challenge.zip

--2025-12-03 13:03:49--  https://zenodo.org/records/17351690/files/imagine_decoding_challenge.zip?download=1
Resolving zenodo.org (zenodo.org)... 2001:1458:d00:17::100:1d9, 2001:1458:d00:24::100:245, 2001:1458:d00:61::100:427, ...
Connecting to zenodo.org (zenodo.org)|2001:1458:d00:17::100:1d9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3011974370 (2.8G) [application/octet-stream]
Saving to: ‘imagine_decoding_challenge.zip’

imagine_decoding_ch 100%[===================>]   2.80G  5.45MB/s    in 10m 3s  

2025-12-03 13:13:53 (4.76 MB/s) - ‘imagine_decoding_challenge.zip’ saved [3011974370/3011974370]



## Imports


In [2]:
import mne
import torch, torchvision
import matplotlib
import numpy as np